In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin"
)

print(mydb)

In [16]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="flowdata"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('netflow',)
('netflows',)
('sites',)


In [40]:
mycursor.execute("ALTER TABLE sites ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")

In [41]:
mycursor.execute("ALTER TABLE sites ADD COLUMN protocol CHAR(5)")

In [15]:
import json
with open('data/netflow/20200528140758.json') as record:
    new_record = json.load(record)
    

In [18]:
new_record

[{'srcaddr': '57.216.112.217',
  'srcport': 63975,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol': 'UDP',
  'flows': 18,
  'packets': 295,
  'bytes': 482341},
 {'srcaddr': '184.105.195.183',
  'srcport': 38885,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol': 'TCP',
  'flows': 3,
  'packets': 112,
  'bytes': 215847},
 {'srcaddr': '125.202.46.124',
  'srcport': 10110,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol': 'UDP',
  'flows': 1,
  'packets': 243,
  'bytes': 115125},
 {'srcaddr': '207.95.145.96',
  'srcport': 17722,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol': 'UDP',
  'flows': 9,
  'packets': 76,
  'bytes': 968991},
 {'srcaddr': '66.239.214.125',
  'srcport': 19071,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol': 'UDP',
  'flows': 3,
  'packets': 388,
  'bytes': 955230},
 {'srcaddr': '232.68.11.247',
  'srcport': 38195,
  'First': 20200528140731,
  'Last': 20200528140931,
  'protocol'

In [19]:
val = [{'57.216.112.217',63975,20200528140731,20200528140931,'UDP',18,295,482341},
 {'184.105.195.183',
   38885,
  20200528140731,
  20200528140931,
   'TCP',
   3,
   112,
  215847}]

val

[{18,
  20200528140731,
  20200528140931,
  295,
  482341,
  '57.216.112.217',
  63975,
  'UDP'},
 {112,
  '184.105.195.183',
  20200528140731,
  20200528140931,
  215847,
  3,
  38885,
  'TCP'}]

In [20]:
sql = "INSERT INTO sites (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"

In [21]:
# Insert one record at a time
for flow in new_record:
    mycursor.execute(sql,(flow["srcaddr"], flow["srcport"],flow["First"],flow["Last"],flow["protocol"],flow["flows"],flow["packets"],flow["bytes"]))
#mycursor.executemany(sql, val)
mydb.commit()    # 数据表内容有更新，必须使用到该语句
 
print(mycursor.rowcount, "记录插入成功。")

1 记录插入成功。


In [51]:
import time

# Insert many records at a time
start_time = time.time()
flows = []
#i = 0
for flow in new_record:
    #i = i+1
    flow_one = (flow["srcaddr"], flow["srcport"], flow["First"], flow["Last"], flow["protocol"], flow["flows"], flow["packets"], flow["bytes"])
    flows.append(flow_one)
    #if(i==3):
       # print(flows)
        #break

mycursor.executemany(sql, flows)
mydb.commit()    # 数据表内容有更新，必须使用到该语句
 

print(mycursor.rowcount, "记录插入成功。")
end_time = time.time()
consume_time = end_time - start_time
consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
print(consume_time) 

1000000 记录插入成功。
00:00:42


In [12]:
mycursor.execute("CREATE TABLE netflow (srcaddr CHAR(50), srcport MEDIUMINT, first BIGINT, last BIGINT, protocol CHAR(5), flows TINYINT, packets SMALLINT, bytes MEDIUMINT, PRIMARY KEY(`srcaddr`))")

In [13]:
# Test: Insert 2 records if it exist, else update
val = [('57.216.112.200',63975,20200528140731,20200528142222,'UDP',13,295,482341),
 ('184.105.195.183',
   38885,
  20200528140731,
  20200528142222,
   'TCP',
   1,
   112,
  215847)]
sql_update = "INSERT INTO netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`"
mycursor.executemany(sql_update, val)
mydb.commit()    # 数据表内容有更新，必须使用到该语句
 
print(mycursor.rowcount, "记录插入成功。")

2 记录插入成功。


In [66]:
mycursor.execute("CREATE TABLE netflows (srcaddr CHAR(50), srcport MEDIUMINT, first BIGINT, last BIGINT, protocol CHAR(5), flows TINYINT, packets SMALLINT, bytes MEDIUMINT, PRIMARY KEY(`srcaddr`))")

In [17]:
import time

# update 1000000 records at a time

sql_update = "INSERT INTO netflows (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

start_time = time.time()
flows = []
#i = 0
for flow in new_record:
    #i = i+1
    flow_one = (flow["srcaddr"], flow["srcport"], flow["First"], flow["Last"], flow["protocol"], flow["flows"], flow["packets"], flow["bytes"])
    flows.append(flow_one)
    #if(i==3):
       # print(flows)
        #break

mycursor.executemany(sql_update, flows)
mydb.commit()    # 数据表内容有更新，必须使用到该语句
 

print(mycursor.rowcount, "记录插入成功。")
end_time = time.time()
consume_time = end_time - start_time
consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
print(consume_time) 

OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 32 Broken pipe

In [1]:
# Python read all files from a directory in order
import os

f = open("./data/netflow_name.txt", 'w')
path = "./data/netflow/"
files = os.listdir(path)
files.sort()

s = []

for file_name in files:
    if not os.path.isdir(path + file_name):
        f_name = str(file_name)
        print(f_name)
        s.append(f_name)
        f.write(f_name + '\n')
for i in s:
    print(i)
print(s)

20200528140758.json
20200528141325.json
20200528141851.json
20200528142416.json
20200528142940.json
20200528143504.json
20200528144029.json
20200528144554.json
20200528145119.json
20200528145643.json
20200528150208.json
20200528150732.json
20200528151257.json
20200528151821.json
20200528152346.json
20200528152910.json
20200528153435.json
20200528153959.json
20200528154524.json
20200528155048.json
20200528155613.json
20200528160137.json
20200528160702.json
20200528161228.json
20200528161752.json
20200528162317.json
20200528162841.json
20200528163405.json
20200528163930.json
20200528164454.json
20200528165019.json
20200528165543.json
20200528170107.json
20200528170632.json
20200528171156.json
20200528171721.json
20200528172246.json
20200528172810.json
20200528173335.json
20200528173859.json
20200528174424.json
20200528174948.json
20200528175513.json
20200528180037.json
20200528180602.json
20200528181126.json
20200528181650.json
20200528182215.json
20200528182740.json
20200528183304.json


In [11]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="flowdata"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('netflows',)


In [5]:
mycursor.execute("CREATE TABLE netflows (srcaddr CHAR(50), srcport MEDIUMINT, first BIGINT, last BIGINT, protocol CHAR(5), flows TINYINT, packets SMALLINT, bytes MEDIUMINT, PRIMARY KEY(`srcaddr`))")

In [14]:
import time
import json

# update 288 X 1000000 records at a time

sql_update = "INSERT INTO netflows (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

for json_name in s:
    mycursor = mydb.cursor()
    start_time = time.time()
    flows = []
    with open('data/netflow/'+json_name) as record:
        new_record = json.load(record)
        for flow in new_record:
            flow_one = (flow["srcaddr"], flow["srcport"], flow["First"], flow["Last"], flow["protocol"], flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor = mydb.cursor()
        mycursor.executemany(sql_update, flows)
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
        mycursor.close()
        print(mycursor.rowcount, "记录插入成功。")
        end_time = time.time()
        consume_time = end_time - start_time
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        print(consume_time) 

-1 记录插入成功。
00:07:45


KeyboardInterrupt: 

In [7]:
mycursor.close()

True

In [8]:
mydb.close()